# Lab 2: Add AgentCore Memory for Personalized Support

In this lab, you'll add **persistent memory** to the customer support agent using **Amazon Bedrock AgentCore Memory**.

## What you'll learn
- How to create AgentCore Memory resources with USER_PREFERENCE and SEMANTIC strategies
- How to integrate memory with Claude Agent SDK using the retrieve/save pattern
- How memory enables personalized, context-aware customer support

## Architecture

<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

## Key Pattern: Manual Retrieve/Save

Unlike Strands Agents (which uses `HookProvider` callbacks), Claude Agent SDK uses explicit memory calls:

```python
# Before query() - retrieve context
context = memory_manager.retrieve_context(user_query)
enhanced_prompt = f"Context:\n{context}\n\n{user_query}"

# Run query()
response = await query(prompt=enhanced_prompt, options=options)

# After query() - save interaction
memory_manager.save_interaction(user_query, response)
```

## Step 1: Setup

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import os
import json
import uuid

os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"
os.environ.pop("CLAUDECODE", None)

import boto3
from boto3.session import Session

boto_session = Session()
REGION = boto_session.region_name
print(f"Region: {REGION}")

## Step 2: Create AgentCore Memory Resource

We create a memory resource with two strategies:
- **USER_PREFERENCE**: Captures customer preferences and behavioral patterns
- **SEMANTIC**: Stores factual information with vector embeddings for retrieval

In [ ]:
from agent.memory_hooks import (
    create_or_get_memory_resource,
    memory_client,
    ACTOR_ID,
    SESSION_ID,
)

# Create or get existing memory resource
memory_id = create_or_get_memory_resource()
print(f"Memory ID: {memory_id}")

## Step 3: Seed Previous Interactions

Let's seed some previous customer interactions so the memory system has data to work with.

In [ ]:
# Seed previous interactions for LTM processing
previous_interactions = [
    {
        "user": "Hi, I'm interested in a laptop for software development. I use Linux.",
        "assistant": "Great choice! For software development on Linux, I'd recommend our laptops with 16GB+ RAM and SSD storage. Our models support Linux with various distributions. Would you like specific model recommendations?"
    },
    {
        "user": "I prefer 15-inch screens and I need at least 32GB RAM.",
        "assistant": "Noted! For a 15-inch laptop with 32GB RAM for development, I'd suggest our Pro series. They come with Intel i7/i9 or AMD Ryzen 9 processors, perfect for compilation and running VMs. Shall I check availability?"
    },
    {
        "user": "What's the warranty coverage if I install Linux instead of Windows?",
        "assistant": "Our hardware warranty remains valid regardless of the operating system you install. The 1-year manufacturer warranty covers all hardware components. However, software support is limited to the pre-installed OS. Would you like details on extended warranty options?"
    }
]

seed_session_id = str(uuid.uuid4())
for interaction in previous_interactions:
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=ACTOR_ID,
        session_id=seed_session_id,
        messages=[
            (interaction["user"], "USER"),
            (interaction["assistant"], "ASSISTANT"),
        ],
    )

print(f"Seeded {len(previous_interactions)} interactions")
print("Waiting for LTM processing (this may take 1-2 minutes)...")

In [ ]:
import time
time.sleep(90)  # Wait for LTM processing
print("LTM processing should be complete.")

## Step 4: Retrieve Processed Memories

In [ ]:
# Check what memories were extracted
strategies = memory_client.get_memory_strategies(memory_id)

for strategy in strategies:
    namespace = strategy["namespaces"][0].format(actorId=ACTOR_ID)
    print(f"\nStrategy: {strategy['type']} | Namespace: {namespace}")
    print("-" * 60)
    
    memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=namespace,
        query="laptop preferences",
        top_k=5,
    )
    
    for i, mem in enumerate(memories):
        if isinstance(mem, dict):
            content = mem.get("content", {}).get("text", "")
            if content:
                print(f"  [{i+1}] {content}")

## Step 5: Initialize Memory Manager and Agent

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions
from agent import prompt_stream
from agent.memory_hooks import CustomerSupportMemoryManager
from agent.mcp_server import get_mcp_server
from agent.prompts import SYSTEM_PROMPT

sdk_server = get_mcp_server()

# Create memory manager
memory_manager = CustomerSupportMemoryManager(
    memory_id=memory_id,
    client=memory_client,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
)


async def invoke_agent_with_memory(user_input: str) -> str:
    """Invoke the agent with memory context retrieval and saving."""
    # Step 1: Retrieve memory context
    context = memory_manager.retrieve_context(user_input)
    enhanced_prompt = f"Customer Context:\n{context}\n\n{user_input}" if context else user_input
    
    print(f"\n--- Memory Context ---")
    print(context if context else "(no context found)")
    print(f"--- End Context ---\n")
    
    # Step 2: Query the agent
    options = ClaudeAgentOptions(
        system_prompt=SYSTEM_PROMPT,
        mcp_servers={"customer-support": sdk_server},
        allowed_tools=["mcp__customer-support__*"],
        permission_mode="bypassPermissions",
        max_turns=10,
    )
    
    response_text = ""
    async for msg in query(prompt=prompt_stream(enhanced_prompt), options=options):
        if hasattr(msg, "result"):
            response_text = msg.result
    
    # Step 3: Save interaction to memory
    memory_manager.save_interaction(user_input, response_text)
    
    return response_text

print("Agent with memory initialized!")

## Step 6: Test Memory-Aware Agent

The agent should now remember the customer's preferences (Linux user, 15-inch, 32GB RAM).

In [ ]:
# Test: The agent should recall customer preferences
response = await invoke_agent_with_memory(
    "Can you recommend a laptop for me? You should know my preferences."
)
print("=" * 60)
print(response)

In [ ]:
# Test: Follow-up question leveraging previous context
response = await invoke_agent_with_memory(
    "What about the warranty on those recommendations?"
)
print("=" * 60)
print(response)

## Summary

In this lab, you added AgentCore Memory to the Claude Agent SDK agent:

1. **Created Memory Resource** with USER_PREFERENCE and SEMANTIC strategies
2. **Seeded interactions** for LTM processing
3. **Built CustomerSupportMemoryManager** with retrieve/save pattern
4. **Tested personalization** - the agent remembers customer preferences

### Key Architecture Difference
- **Strands**: Uses `HookProvider` with event callbacks (automatic)
- **Claude Agent SDK**: Uses explicit `retrieve_context()` before and `save_interaction()` after `query()`

In **Lab 3**, we'll add AgentCore Gateway to centralize and share tools across multiple agents.